<a href="https://colab.research.google.com/github/Priya29satya07/miniproject_1/blob/main/Copy_of_newcnn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense
from tensorflow.keras.layers import Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import l2

In [3]:
from tensorflow.keras import backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Data Preparation

batch_size = 32

datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/covid-qu-ex dataset/Infection Segmentation Data/Infection Segmentation Data/Train',
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/covid-qu-ex dataset/Infection Segmentation Data/Infection Segmentation Data/Val',
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)



Found 8964 images belonging to 3 classes.
Found 557 images belonging to 3 classes.


In [7]:


# Directory where the checkpoints will be saved
checkpoint_dir = 'Custom_Model/'
os.makedirs(checkpoint_dir, exist_ok=True)

# Define a ModelCheckpoint callback to save the model at every improved epoch
checkpoint = ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_epoch_{epoch:02d}_train_acc_{accuracy:.4f}_val_acc_{val_accuracy:.4f}.h5'),
    monitor='val_accuracy',  # metric to monitor
    verbose=1,  # verbosity - 0 or 1
    save_best_only=True,  # Save only the best model according to the monitored metric
    mode='max',  # Since we are monitoring accuracy, higher is better
    save_weights_only=False  # Save the full model, not just the weights
)

# Create a custom CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(256, 256, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (1, 1), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu', name='fc1'))
model.add(Dense(32, activation='relu', name='fc2'))
model.add(Dense(3, activation='softmax'))  # Assuming 3 classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1_score])

# Adjust the `workers` parameter as necessary
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[checkpoint],
    use_multiprocessing=True,  # Enable multiprocessing
    workers=6  # Number of workers to use. Adjust as per your CPU cores.
)


Epoch 1/50
281/281 [==============================] - ETA: 0s - loss: 0.8528 - accuracy: 0.5833 - f1_score: 0.4963
Epoch 1: val_accuracy improved from -inf to 0.50090, saving model to Custom_Model/model_epoch_01_train_acc_0.5833_val_acc_0.5009.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


281/281 [==============================] - 481s 2s/step - loss: 0.8528 - accuracy: 0.5833 - f1_score: 0.4963 - val_loss: 1.0310 - val_accuracy: 0.5009 - val_f1_score: 0.0692
Epoch 2/50
281/281 [==============================] - ETA: 0s - loss: 0.7453 - accuracy: 0.6283 - f1_score: 0.5738
Epoch 2: val_accuracy did not improve from 0.50090
281/281 [==============================] - 191s 658ms/step - loss: 0.7453 - accuracy: 0.6283 - f1_score: 0.5738 - val_loss: 1.0909 - val_accuracy: 0.4596 - val_f1_score: 0.0000e+00
Epoch 3/50
281/281 [==============================] - ETA: 0s - loss: 0.7011 - accuracy: 0.6570 - f1_score: 0.6255
Epoch 3: val_accuracy improved from 0.50090 to 0.55296, saving model to Custom_Model/model_epoch_03_train_acc_0.6570_val_acc_0.5530.h5
281/281 [==============================] - 192s 672ms/step - loss: 0.7011 - accuracy: 0.6570 - f1_score: 0.6255 - val_loss: 0.9718 - val_accuracy: 0.5530 - val_f1_score: 0.3945
Epoch 4/50
281/281 [==============================] 

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 252, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 126, 126, 32)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 122, 122, 64)      51264     
                                                                 
 conv2d_3 (Conv2D)           (None, 118, 118, 64)      102464    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 59, 59, 64)        0         
 g2D)                                                   